In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image

In [2]:
scyjava.config.add_option('-Xmx30g')
start_dir = os.getcwd()
#ij = imagej.init('sc.fiji:fiji', mode='interactive')
ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
## Something about this init() function changes the current working directory.
ij.getApp().getInfo(True)
ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number

'2.9.0/1.54e'

X
Y
Z
X
Y
Z
X
Y
Z


In [3]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
ov = Overlay()

In [4]:
Overlay = scyjava.jimport('ij.gui.Overlay')
ov = Overlay()
rm = ij.RoiManager.getRoiManager()

In [3]:
directory_path = '/home/saka/Documents/Lab stuff/confocal/exp2'
#directory_path = '/home/saka/Documents/Zhihui/timelapse/images'
os.chdir(directory_path)

In [6]:
root = tk.Tk()
root.withdraw()
filepath = filedialog.askopenfilename()

In [31]:
image = ij.io().open(filepath)
#ij.ui().show(image)
imp = ij.py.to_imageplus(image)

In [32]:
ij.ui().show(imp)

In [ ]:


#wanted_channel = 2
#blue_channel = image[:, :, wanted_channel,:,:]
#imp = ij.py.to_imageplus(blue_channel)



In [9]:
wanted_channel = 3
channel = image[:, :, wanted_channel, :, :]
imp = ij.py.to_imageplus(channel)
imp.setDimensions(1, 11, 121)
#ij.ui().show(imp)
ZProjector = scyjava.jimport("ij.plugin.ZProjector")()
method = 'sum all'
z_projector_result = ZProjector.run(imp, method)
z_collapsed_image = ij.py.from_java(z_projector_result)
z_collapsed_dataset = ij.py.to_dataset(z_collapsed_image)
ij.ui().show(z_collapsed_dataset)

In [20]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_frame = data_info['Z']

In [36]:
file_prefix = "Image_T"
for timepoint in range(1, num_frame + 1):
    frame_number = f"{timepoint:04d}"
    #input_txt = Path(f"{directory_path}/txt_outlines/Experiment-1647-Split Scenes-01_z2/cellpose/frame_{frame_number}_cp_outlines.txt")
    input_txt = Path(f"{directory_path}/txt_outlines/{file_prefix}{frame_number}_Overlay_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
            roi_count = rm.getCount()
            rm.select(roi_count - 1)
            time_set = imp.setZ(timepoint)   
            rm.runCommand('Update')
ij.py.run_macro("roiManager('Select All');")
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0]
rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip")

True

In [ ]:
ij.ui().show(image)

In [4]:
file_path = Path(f"{directory_path}/results_bacteria.csv")
df = pandas.read_csv(file_path)
#df = pandas.read_csv(dataset)


In [5]:
selected_column = df.iloc[:, 1]
new_column_name = "ROI_ID"
df = df.rename(columns = selected_column : new_column_name)

SyntaxError: invalid syntax (4029938479.py, line 3)

In [26]:
selected_column = df.iloc[:, 1]
selected_column.rename(new_column_name)

0      3.492
1      2.344
2      1.736
3      0.530
4      2.619
       ...  
634    0.118
635    0.069
636    1.246
637    0.137
638    0.726
Name: ROI_ID, Length: 639, dtype: float64

In [22]:
df.head()

,,Area,Mean,Min,Max,X,Y,IntDen,Median,Skew,Kurt,RawIntDen,Slice,Frame
0,1,3.492,255,255,255,18.828,14.062,890.379,255,NaN,NaN,90780,1,2
1,2,2.344,255,255,255,18.652,14.875,597.755,255,NaN,NaN,60945,2,2
2,3,1.736,255,255,255,18.640,15.192,442.689,255,NaN,NaN,45135,3,2
3,4,0.530,255,255,255,17.707,17.084,135.058,255,NaN,NaN,13770,3,3
4,5,2.619,255,255,255,17.929,16.364,667.784,255,NaN,NaN,68085,4,3


In [30]:
x_column='X'
y_column='Y'
z_column='Slice'
gdf = geopandas.GeoDataFrame(
        df, geometry=geopandas.points_from_xy(df[x_column], df[y_column], df[z_column]))

In [31]:
final_time = gdf.Frame.max()

In [ ]:
pairwise_distances = []
    for start_time in range(0, final_time):
        i = start_time
        j = i + 1
        ti_idx = gdf.Frame == i
        tj_idx = gdf.Frame == j